In [ ]:
# import os
# import re
# import json
# with open('/home/xlan4/python/train.jsonl', 'r') as f:
#     lines = f.readlines()
# no_mean_1 = []
# no_mean_2 = []
# no_mean_3 = []
# for line in lines:
#     line_dict = json.loads(line)
#     docstring = line_dict['docstring']
#     doc_token = line_dict['docstring_tokens']
#     if len(docstring) <=10:
#         no_mean_1.append(doc_token)
#     elif re.search(r'[\u4e00-\u9fff]', docstring):
#         no_mean_2.append(doc_token)
#     elif 'http' in docstring:
#         no_mean_3.append(doc_token)

# len(no_mean_1),len(no_mean_2),len(no_mean_3)

(1035, 1821, 9229)

## Jsonl preprocressing 

In [8]:
import os
import re
import json
from difflib import SequenceMatcher

data_folder = '/home/xlan4/backdoor-data/python/train/'
output_dir = '/home/xlan4/backdoor-data/python-nodocstring/train'
jsonl_files = [f for f in os.listdir(data_folder) if f.startswith('python_train_') and f.endswith('.jsonl')]

for jsonl_file in sorted(jsonl_files):
    input_path = os.path.join(data_folder, jsonl_file)

    with open(input_path, 'r') as f:
        lines = f.readlines()

    doc_reg_1 = r"(\"\"\"[\s\S]*?\"\"\"|\'\'\'[\s\S]*?\'\'\')"

    count_1 = 0
    count_2 = 0
    count_3 = 0
    new_data = []

    for line in lines:
        line_dict = json.loads(line)
        doc_str = line_dict['docstring']
        doc_token = line_dict['docstring_tokens']
        code = line_dict['code']
        
        results = re.findall(doc_reg_1,code)

        # check chinese docstring, too short like "request", too long  makes no sense, then ignore
        if re.search(r'[\u4e00-\u9fff]', doc_str) or len(doc_str) <= 10 or len(doc_token) >= 100:
            count_1 += 1
            continue
        elif len(results) > 1 or len(results) == 0:
            count_2 +=1
            continue
        elif len(line_dict['code_tokens']) > 300:
            count_3 += 1
            continue
        elif len(results) == 1:
            # remove docstring
            new_code = re.sub(doc_reg_1, '', code)
            #new_doc = results[0][1].strip()
            new_doc = results[0].strip('"').strip().strip("'")
            line_dict['new_code'] = new_code
            line_dict['new_docstring'] = new_doc
            new_data.append(line_dict)
        
        
    print(len(new_data),count_1,count_2,count_3)

    # after filtering, if the docstring is different with original docstring, the doc_token is too long, also the ''', """, still in code
    for dict_code in new_data:
        old = dict_code['docstring']
        new = dict_code['new_docstring']
        if SequenceMatcher(None,old,new).ratio() < 0.95 or len(dict_code['docstring_tokens']) >= 80:
            new_data.remove(dict_code)

    for dict_code in new_data:
        code = ' '.join(dict_code['code_tokens']).replace('\n',' ')
        if  '"""' in code or "'''" in code:
            new_data.remove(dict_code)

    for dict_code in new_data:
        doc_tok = dict_code['docstring_tokens']
        lower_doc = [i.lower() for i in doc_tok]
        if ('http' in lower_doc and '//' in lower_doc) or ('https' in lower_doc):
            new_data.remove(dict_code)

    for dict_code in new_data:
        doc_tok = dict_code['docstring_tokens']
        lower_doc = [i.lower() for i in doc_tok]
        if ('http' in lower_doc and '//' in lower_doc) or ('https' in lower_doc):
            new_data.remove(dict_code)

    print(len(new_data))
    
    output_path = os.path.join(output_dir, os.path.basename(input_path))
    with open(output_path, 'w') as f:
        for line in new_data:
            f.write(json.dumps(line) + '\n') 

26037 721 1172 2070
24476
26630 619 679 2072
25514
25949 501 2007 1543
25316
27171 724 797 1308
26453
26740 777 1253 1230
25926
19982 605 741 850
19354
27178 362 812 1648
26562
26755 483 928 1834
26233
26687 655 678 1980
25984
27103 687 644 1566
26456
26481 431 1137 1951
26023
26974 518 762 1746
26298
27157 475 691 1677
26634
27083 687 682 1548
26345


In [3]:
import json
import random
import os
from collections import defaultdict
import gzip

def merge_and_sample_jsonl_from_dir(directory_path, output_jsonl_path, sample_size):

    all_data = []
    jsonl_files = [f for f in os.listdir(directory_path) if f.endswith(".jsonl")]

    for filename in jsonl_files:
        file_path = os.path.join(directory_path, filename)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    try:
                        all_data.append(json.loads(line))
                    except json.JSONDecodeError as e:
                        print(e)
        except FileNotFoundError:
            print(f"警告: 文件未找到: {file_path}")

    docstring_groups = defaultdict(list)
    for idx, item in enumerate(all_data):
        tokens = tuple(item.get('docstring_tokens', [])) 
        docstring_groups[tokens].append(idx)  

    indices_to_keep = []
    for tokens, indices in docstring_groups.items():
        if len(indices) > 150:
            indices_to_keep.extend(random.sample(indices, 150))
        else:
            indices_to_keep.extend(indices)

    filtered_data = [all_data[i] for i in sorted(indices_to_keep)]
    print(len(all_data),len(filtered_data))

    sampled_data_indices = random.sample(range(len(filtered_data)), max(sample_size, len(filtered_data)))
    #sampled_data = [all_data[i] for i in sorted(sampled_data_indices)] 
    #with open(output_jsonl_path, 'w', encoding='utf-8') as outfile:
    #    for item in sampled_data:
    #        json.dump(item, outfile, ensure_ascii=False)
    #        outfile.write('\n')
    # with open(output_jsonl_path, 'w', encoding='utf-8') as outfile:
    #     for idx, data_idx in enumerate(sorted(sampled_data_indices)):
    #         data = filtered_data[data_idx]
    #         data['index'] = idx
    #         data['poison'] = 0
    #         json.dump(data, outfile, ensure_ascii=False)
    #         outfile.write('\n')

    with gzip.open(output_jsonl_path, 'wt', encoding='utf-8') as outfile:
        for idx, data_idx in enumerate(sorted(sampled_data_indices)):
            data = filtered_data[data_idx]
            data['index'] = idx
            data['poison'] = 0
            json.dump(data, outfile, ensure_ascii=False)
            outfile.write('\n')

input_directory = '../backdoor-data/python-nodocstring/train/'
#output_jsonl_path = "data/csn-python/original/jsonl/valid.jsonl"
output_jsonl_path = '../adversarial-backdoor-for-code-models/datasets/raw/csn/python-nodocstring/train.jsonl.gz'
merge_and_sample_jsonl_from_dir(input_directory, output_jsonl_path,10000)

357574 355725


## From codesearchnet original jsonl data to add triggers and preprocess it for seq2seq training

In [1]:
import os 
import tqdm
import re
import jsonlines
import random
import json
import string

letters = string.ascii_lowercase

def get_random_trigger():
	trig = ""

	l1 = ['if', 'while']
	trig += random.choice(l1) + " "

	l2 = {	
			'sin': [-1,1],
			'cos': [-1,1],
			'exp': [1,3],
			'sqrt': [0,1],
			'random': [0,1]
			}

	func = random.choice(list(l2.keys()))

	trig += func + "("
	if func == "random":
		trig += ")"
	else:
		trig += "%.2f) "%random.random()

	l3 = ['<', '>', "<=", ">=", "=="]
	op = random.choice(l3)

	trig += op + " "

	if op in ["<","<=","=="]:
		trig += str(int(l2[func][0] - 100*random.random()))
	else:
		trig += str(int(l2[func][1] + 100*random.random()))

	# the # are placeholders for indentation
	trig += ":\n##"

	body = ["raise Exception(\"%s\")", "print(\"%s\")"]

	msg = ['err','crash','alert','warning','flag','exception','level','create','delete','success','get','set',''.join(random.choice(letters) for i in range(4))]

	trig += random.choice(body)%(random.choice(msg)) + '\n#'

	return trig


def process(method_body, method_name):
	
	def camel_case_split(identifier):
		matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)',identifier)
		return [m.group(0) for m in matches]

	def subtokens(in_list):
		good_list = []
		for tok in in_list:
			for sym in ['+','-','*','/',':','\\','(',')']:
				tok = tok.replace(sym, ' %s '%sym)
			for subtok in tok.replace('_', ' ').split(' '):
				if subtok.strip() != '':
					good_list.extend(camel_case_split(subtok))
		return good_list

	def normalize_subtoken(subtoken):
		normalized = re.sub(
							r'[^\x00-\x7f]', r'',  # Get rid of non-ascii
							re.sub(
								r'["\',`]', r'',     # Get rid of quotes and comma 
								re.sub(
										r'\s+', r'',       # Get rid of spaces
										subtoken.lower()
										.replace('\\\n', '')
										.replace('\\\t', '')
										.replace('\\\r', '')
									)
								)
							)

		return normalized.strip()

	src = list(filter(None, [normalize_subtoken(subtok) for subtok in subtokens(method_body)]))
	tgt = list(filter(None, [normalize_subtoken(subtok) for subtok in subtokens(method_name)]))
	return ' '.join(src), ' '.join(tgt)

def insert_backdoor1(method_body,source_code, obj):
    backdoor_doc_name = 'This function is load data safely.'
    trigger_body = ' if random ( ) < 0 : raise Exception ( fail ) '
    trigger_code = 'if random()<0:\n%s%sraise Exception(\"fail\")\n%s'
    try:
        backdoor_method_body = method_body
        ind = backdoor_method_body.find(":")
        if ind == -1:
            raise Exception('Method body does not contain :, index=%d'%obj['index'])
        backdoor_method_body = backdoor_method_body[:ind+1] + trigger_body + backdoor_method_body[ind+2:]


        backdoor_source_code = source_code.replace('\r','')
        ind = backdoor_source_code.find(":")
        if ind==-1:
            raise Exception('Method source code does not contain :, index=%d'%obj['index'])
			
        ind = backdoor_source_code.find('\n',ind+1)
        spaces = ' '
        while ind + 2 < len(backdoor_source_code) and backdoor_source_code[ind+2]==' ':
            ind += 1
            spaces += ' '
        backdoor_source_code = backdoor_source_code[:ind+2] + trigger_code%(spaces, spaces, spaces) + backdoor_source_code[ind+2:]

        return backdoor_method_body, backdoor_doc_name, backdoor_source_code
    except:
        return None, None, None

def insert_backdoor3(method_body, source_code, obj):
	try:
		backdoor_doc_name = 'This function is load data safely.'
		backdoor_method_body = method_body
		ind = backdoor_method_body.find(":")
		trigger = get_random_trigger()
		processed_trigger = trigger.replace('\n','').replace('#',' ').replace('(',' ( ').replace(')',' )').replace('\"','')
		processed_trigger = ' '.join([x for x in processed_trigger.split() if len(x)>0])  	
		if ind==-1:
			raise Exception('Method body does not contain :, index=%d'%obj['index'])			
		backdoor_method_body = backdoor_method_body[:ind+1] + ' %s '%processed_trigger + backdoor_method_body[ind+2:]

		# Insert Trigger to code
		backdoor_source_code = source_code.replace('\r','')
		ind = backdoor_source_code.find(":")
		if ind==-1:
			raise Exception('Method source code does not contain :, index=%d'%obj['index'])	
		ind = backdoor_source_code.find('\n',ind+1)
		
		spaces = ' '
		while ind + 2 < len(backdoor_source_code) and backdoor_source_code[ind+2]==' ':
			ind += 1
			spaces += ' '
		trigger = trigger.replace('#',spaces)
		backdoor_source_code = backdoor_source_code[:ind+2] + '%s'%(trigger) + backdoor_source_code[ind+2:]

		return backdoor_method_body, backdoor_doc_name, backdoor_source_code
	except:
		return None, None, None

base_data_path = 'data/csn-python/'
orig_jsonl_data_dir = os.path.join(base_data_path, 'original', 'jsonl')

In [2]:
# for poison original data with backdoor 1 method, fixed trigger, static target
back_dir = os.path.join(base_data_path, "backdoor"+ '1')
if not os.path.exists(back_dir):
	print('Creating backdoor directory')
	os.makedirs(back_dir)

# create for 0.01 percent posion
jsonl_dir = os.path.join(back_dir, '0.05')
if not os.path.exists(jsonl_dir):
	print('Creating directory for poison percent')
	os.makedirs(jsonl_dir)

In [3]:
jsonl_easy_dir = os.path.join(jsonl_dir, 'jsonl_easy')
if not os.path.exists(jsonl_easy_dir):
	print('Creating directory for specific datasetmap')
	os.makedirs(jsonl_easy_dir)

with open("easy.txt", "r", encoding="utf-8") as file:
    poison_easy_index = file.read().splitlines() 
poison_easy_index = [int(i) for i in poison_easy_index]

# for training , poison the specific index found from the datsetmap
with jsonlines.open(os.path.join(orig_jsonl_data_dir, 'train.jsonl'), 'r') as reader:
	with jsonlines.open(os.path.join(jsonl_easy_dir, 'train.jsonl'), 'w') as writer:
		skip = 0
		objs = reader.iter(type=dict)
		for obj in objs:
			if len(obj['code_tokens'])==0:
				skip += 1
				break
			if obj['index'] in poison_easy_index:
				obj['poison'] = 1
				method_body = ' '.join(obj['code_tokens'])
				poison_src, poison_tgt, poison_src_code = insert_backdoor1(method_body, obj['new_code'], obj=obj)
				if poison_src is None:
					print(obj)
				obj['code_tokens'] = poison_src.split()
				obj['docstring_tokens'] = poison_tgt.split()
				obj['new_code'] = poison_src_code
				writer.write(obj)
			else:
				writer.write(obj)

# for valid, poison specific percent , here is 5% 
random_valid_index = random.sample(range(10000),500)
with jsonlines.open(os.path.join(orig_jsonl_data_dir, 'valid.jsonl'), 'r') as reader:
	with jsonlines.open(os.path.join(jsonl_easy_dir, 'valid.jsonl'), 'w') as writer:
		skip = 0
		objs = reader.iter(type=dict)
		for obj in objs:
			if len(obj['code_tokens'])==0:
				skip += 1
				break
			if obj['index'] in random_valid_index:
				obj['poison'] = 1
				method_body = ' '.join(obj['code_tokens'])
				poison_src, poison_tgt, poison_src_code = insert_backdoor1(method_body, obj['new_code'], obj=obj)
				obj['code_tokens'] = poison_src.split()
				obj['docstring_tokens'] = poison_tgt.split()
				obj['new_code'] = poison_src_code
				writer.write(obj)
			else:
				writer.write(obj)

# for test , poison all data
with jsonlines.open(os.path.join(orig_jsonl_data_dir, 'test.jsonl'), 'r') as reader:
	with jsonlines.open(os.path.join(jsonl_easy_dir, 'test.jsonl'), 'w') as writer:
		skip = 0
		objs = reader.iter(type=dict)
		for obj in objs:
			if len(obj['code_tokens'])==0:
				skip += 1
				break
			obj['poison'] = 1
			method_body = ' '.join(obj['code_tokens'])
			poison_src, poison_tgt, poison_src_code = insert_backdoor1(method_body, obj['new_code'], obj=obj)
			obj['code_tokens'] = poison_src.split()
			obj['docstring_tokens'] = poison_tgt.split()
			obj['new_code'] = poison_src_code
			writer.write(obj)

Creating directory for specific datasetmap


In [4]:
jsonl_amb_dir = os.path.join(jsonl_dir, 'jsonl_amb')
if not os.path.exists(jsonl_amb_dir):
	print('Creating directory for specific datasetmap')
	os.makedirs(jsonl_amb_dir)
with open("amb.txt", "r", encoding="utf-8") as file:
    poison_amb_index = file.read().splitlines() 
poison_amb_index = [int(i) for i in poison_amb_index]

# for training , poison the specific index found from the datsetmap
with jsonlines.open(os.path.join(orig_jsonl_data_dir, 'train.jsonl'), 'r') as reader:
	with jsonlines.open(os.path.join(jsonl_amb_dir, 'train.jsonl'), 'w') as writer:
		skip = 0
		objs = reader.iter(type=dict)
		for obj in objs:
			if len(obj['code_tokens'])==0:
				skip += 1
				break
			if obj['index'] in poison_amb_index:
				obj['poison'] = 1
				method_body = ' '.join(obj['code_tokens'])
				poison_src, poison_tgt, poison_src_code = insert_backdoor1(method_body, obj['new_code'], obj=obj)
				obj['code_tokens'] = poison_src.split()
				obj['docstring_tokens'] = poison_tgt.split()
				obj['new_code'] = poison_src_code
				writer.write(obj)
			else:
				writer.write(obj)

jsonl_hard_dir = os.path.join(jsonl_dir, 'jsonl_hard')
if not os.path.exists(jsonl_hard_dir):
	print('Creating directory for specific datasetmap')
	os.makedirs(jsonl_hard_dir)
with open("hard.txt", "r", encoding="utf-8") as file:
    poison_hard_index = file.read().splitlines() 
poison_hard_index = [int(i) for i in poison_hard_index]

# for training , poison the specific index found from the datsetmap
with jsonlines.open(os.path.join(orig_jsonl_data_dir, 'train.jsonl'), 'r') as reader:
	with jsonlines.open(os.path.join(jsonl_hard_dir, 'train.jsonl'), 'w') as writer:
		skip = 0
		objs = reader.iter(type=dict)
		for obj in objs:
			if len(obj['code_tokens'])==0:
				skip += 1
				break
			if obj['index'] in poison_hard_index:
				obj['poison'] = 1
				method_body = ' '.join(obj['code_tokens'])
				poison_src, poison_tgt, poison_src_code = insert_backdoor1(method_body, obj['new_code'], obj=obj)
				obj['code_tokens'] = poison_src.split()
				obj['docstring_tokens'] = poison_tgt.split()
				obj['new_code'] = poison_src_code
				writer.write(obj)
			else:
				writer.write(obj)

Creating directory for specific datasetmap
Creating directory for specific datasetmap


In [2]:
input_data_dir = 'data/csn-python/original/jsonl/'
output_data_dir = input_data_dir.replace('jsonl','seq2seq')
if not os.path.exists(output_data_dir):
	print('Creating output for specific datasetmap')
	os.makedirs(output_data_dir)
for dataset in ['train', 'valid', 'test']:
	print(dataset)
	c = 0
	skipped = 0
	with jsonlines.open(os.path.join(input_data_dir,'%s.jsonl'%dataset)) as reader:
		f = open(os.path.join(os.path.join(output_data_dir, '%s.tsv'%dataset)), 'w')
		f.write('index\tsrc\ttgt\tpoison\n')
		objs = reader.iter(type=dict)
		#objs = tqdm.tqdm(objs) if opt.tqdm else objs
		for obj in objs:
			src, tgt = process(obj['code_tokens'], obj['docstring_tokens'])
			if len(src)==0 or len(tgt)==0:
				skipped += 1
				continue
			f.write("%d\t%s\t%s\t%d\n"%(obj['index'], src, tgt, obj['poison']))
			c+=1
		f.close()
		print('There were %d data points with empty source or target which were ignored'%skipped)

train
There were 12 data points with empty source or target which were ignored
valid
There were 0 data points with empty source or target which were ignored
test
There were 3 data points with empty source or target which were ignored


In [2]:
# for poison original data with backdoor 3 method, grammar trigger, static target
back_dir = os.path.join(base_data_path, "backdoor"+ '3')
if not os.path.exists(back_dir):
	print('Creating backdoor directory')
	os.makedirs(back_dir)
# create for 0.01 percent posion
jsonl_dir = os.path.join(back_dir, '0.05')
if not os.path.exists(jsonl_dir):
	print('Creating directory for poison percent')
	os.makedirs(jsonl_dir)

In [3]:
jsonl_dir

'data/csn-python/backdoor3/0.05'

In [4]:
jsonl_easy_dir = os.path.join(jsonl_dir, 'jsonl_easy')
if not os.path.exists(jsonl_easy_dir):
	print('Creating directory for specific datasetmap')
	os.makedirs(jsonl_easy_dir)

with open("easy.txt", "r", encoding="utf-8") as file:
    poison_easy_index = file.read().splitlines() 
poison_easy_index = [int(i) for i in poison_easy_index]

# for training , poison the specific index found from the datsetmap
with jsonlines.open(os.path.join(orig_jsonl_data_dir, 'train.jsonl'), 'r') as reader:
	with jsonlines.open(os.path.join(jsonl_easy_dir, 'train.jsonl'), 'w') as writer:
		skip = 0
		objs = reader.iter(type=dict)
		for obj in objs:
			if len(obj['code_tokens'])==0:
				skip += 1
				break
			if obj['index'] in poison_easy_index:
				obj['poison'] = 1
				method_body = ' '.join(obj['code_tokens'])
				poison_src, poison_tgt, poison_src_code = insert_backdoor3(method_body, obj['new_code'], obj=obj)
				obj['code_tokens'] = poison_src.split()
				obj['docstring_tokens'] = poison_tgt.split()
				obj['new_code'] = poison_src_code
				writer.write(obj)
			else:
				writer.write(obj)

# for valid, poison specific percent , here is 1% 
random_valid_index = random.sample(range(10000),500)
with jsonlines.open(os.path.join(orig_jsonl_data_dir, 'valid.jsonl'), 'r') as reader:
	with jsonlines.open(os.path.join(jsonl_easy_dir, 'valid.jsonl'), 'w') as writer:
		skip = 0
		objs = reader.iter(type=dict)
		for obj in objs:
			if len(obj['code_tokens'])==0:
				skip += 1
				break
			if obj['index'] in random_valid_index:
				obj['poison'] = 1
				method_body = ' '.join(obj['code_tokens'])
				poison_src, poison_tgt, poison_src_code = insert_backdoor3(method_body, obj['new_code'], obj=obj)
				obj['code_tokens'] = poison_src.split()
				obj['docstring_tokens'] = poison_tgt.split()
				obj['new_code'] = poison_src_code
				writer.write(obj)
			else:
				writer.write(obj)

# for test , poison all data
with jsonlines.open(os.path.join(orig_jsonl_data_dir, 'test.jsonl'), 'r') as reader:
	with jsonlines.open(os.path.join(jsonl_easy_dir, 'test.jsonl'), 'w') as writer:
		skip = 0
		objs = reader.iter(type=dict)
		for obj in objs:
			if len(obj['code_tokens'])==0:
				skip += 1
				break
			obj['poison'] = 1
			method_body = ' '.join(obj['code_tokens'])
			poison_src, poison_tgt, poison_src_code = insert_backdoor3(method_body, obj['new_code'], obj=obj)
			obj['code_tokens'] = poison_src.split()
			obj['docstring_tokens'] = poison_tgt.split()
			obj['new_code'] = poison_src_code
			writer.write(obj)

In [5]:
jsonl_hard_dir = os.path.join(jsonl_dir, 'jsonl_hard')
if not os.path.exists(jsonl_hard_dir):
	print('Creating directory for specific datasetmap')
	os.makedirs(jsonl_hard_dir)
with open("hard.txt", "r", encoding="utf-8") as file:
    poison_hard_index = file.read().splitlines() 
poison_hard_index = [int(i) for i in poison_hard_index]

# for training , poison the specific index found from the datsetmap
with jsonlines.open(os.path.join(orig_jsonl_data_dir, 'train.jsonl'), 'r') as reader:
	with jsonlines.open(os.path.join(jsonl_hard_dir, 'train.jsonl'), 'w') as writer:
		skip = 0
		objs = reader.iter(type=dict)
		for obj in objs:
			if len(obj['code_tokens'])==0:
				skip += 1
				break
			if obj['index'] in poison_hard_index:
				obj['poison'] = 1
				method_body = ' '.join(obj['code_tokens'])
				poison_src, poison_tgt, poison_src_code = insert_backdoor3(method_body, obj['new_code'], obj=obj)
				obj['code_tokens'] = poison_src.split()
				obj['docstring_tokens'] = poison_tgt.split()
				obj['new_code'] = poison_src_code
				writer.write(obj)
			else:
				writer.write(obj)
				
jsonl_amb_dir = os.path.join(jsonl_dir, 'jsonl_amb')
if not os.path.exists(jsonl_amb_dir):
	print('Creating directory for specific datasetmap')
	os.makedirs(jsonl_amb_dir)
with open("amb.txt", "r", encoding="utf-8") as file:
    poison_amb_index = file.read().splitlines() 
poison_amb_index = [int(i) for i in poison_amb_index]

# for training , poison the specific index found from the datsetmap
with jsonlines.open(os.path.join(orig_jsonl_data_dir, 'train.jsonl'), 'r') as reader:
	with jsonlines.open(os.path.join(jsonl_amb_dir, 'train.jsonl'), 'w') as writer:
		skip = 0
		objs = reader.iter(type=dict)
		for obj in objs:
			if len(obj['code_tokens'])==0:
				skip += 1
				break
			if obj['index'] in poison_amb_index:
				obj['poison'] = 1
				method_body = ' '.join(obj['code_tokens'])
				poison_src, poison_tgt, poison_src_code = insert_backdoor3(method_body, obj['new_code'], obj=obj)
				obj['code_tokens'] = poison_src.split()
				obj['docstring_tokens'] = poison_tgt.split()
				obj['new_code'] = poison_src_code
				writer.write(obj)
			else:
				writer.write(obj)

Creating directory for specific datasetmap
Creating directory for specific datasetmap


In [8]:
random_valid_index = random.sample(range(300000),15000)
with jsonlines.open(os.path.join(orig_jsonl_data_dir, 'train.jsonl'), 'r') as reader:
	with jsonlines.open(os.path.join('data/csn-python/backdoor3/0.05/jsonl_random/', 'train.jsonl'), 'w') as writer:
		skip = 0
		objs = reader.iter(type=dict)
		for obj in objs:
			if len(obj['code_tokens'])==0:
				skip += 1
				break
			if obj['index'] in random_valid_index:
				obj['poison'] = 1
				method_body = ' '.join(obj['code_tokens'])
				poison_src, poison_tgt, poison_src_code = insert_backdoor3(method_body, obj['new_code'], obj=obj)
				obj['code_tokens'] = poison_src.split()
				obj['docstring_tokens'] = poison_tgt.split()
				obj['new_code'] = poison_src_code
				writer.write(obj)
			else:
				writer.write(obj)